## Football API testing 
https://rapidapi.com/api-sports/api/api-football/details

In [1]:
import os 
import requests
import json

Using conda to manage my `api_key` and `api_host`. How to do this can be [here](https://towardsdatascience.com/how-to-hide-your-api-keys-in-python-fb2e1a61b0a0). 

In [4]:
url = "https://api-football-v1.p.rapidapi.com/v2/predictions/157462"

headers = {
    'x-rapidapi-host': os.environ.get("api_host"), 
    'x-rapidapi-key': os.environ.get("api_key"), 
    }

An example call using the `requests` library to pull the json response from the given link. Carrying and storing IDs will allow for pulling all the information for a given league within a single league. 

In [6]:
response = requests.request("GET", url, headers=headers)

In [11]:
json.loads(response.text)

{'api': {'results': 1,
  'predictions': [{'match_winner': '1',
    'under_over': None,
    'goals_home': '-3.5',
    'goals_away': '-1.5',
    'advice': 'Winner : Paris Saint Germain',
    'winning_percent': {'home': '50%', 'draws': '50%', 'away': '0%'},
    'teams': {'home': {'team_id': 85,
      'team_name': 'Paris Saint Germain',
      'last_5_matches': {'forme': '75%',
       'att': '71%',
       'def': '86%',
       'goals': 10,
       'goals_avg': 2.5,
       'goals_against': 2,
       'goals_against_avg': 0.5},
      'all_last_matches': {'matchs': {'matchsPlayed': {'home': 2,
         'away': 2,
         'total': 4},
        'wins': {'home': 2, 'away': 1, 'total': 3},
        'draws': {'home': 0, 'away': 0, 'total': 0},
        'loses': {'home': 0, 'away': 1, 'total': 1}},
       'goals': {'goalsFor': {'home': 7, 'away': 3, 'total': 10},
        'goalsAgainst': {'home': 0, 'away': 2, 'total': 2}},
       'goalsAvg': {'goalsFor': {'home': '3.5', 'away': '1.5', 'total': '2.5'},
  

In [12]:
base_url = 'https://api-football-v1.p.rapidapi.com/v2/'
leagues_url = base_url + 'leagues/'
leagues_url

'https://api-football-v1.p.rapidapi.com/v2/leagues/'

In [21]:
def api_call(url):
    response = requests.request("GET", url, headers=headers)
    # Return a dictionary of the response text
    return json.loads(response.text)

In [22]:
available_leagues = api_call(leagues_url)

In [26]:
import pandas as pd

In [28]:
leagues_df = pd.DataFrame.from_dict(available_leagues['api']['leagues'])

In [54]:
premier_league_id = leagues_df[(leagues_df.country == 'England') & (leagues_df.is_current == 1) & (leagues_df.name == 'Premier League')].league_id
premier_league_id.values[0]

524

### API Relationships
__Fixture__ - need {league_id}  
__Events__ - need {fixture_id}

In [55]:
league_fixtures = base_url + 'fixtures/league/' + str(premier_league_id.values[0])
league_fixtures

'https://api-football-v1.p.rapidapi.com/v2/fixtures/league/524'

In [57]:
league_fixtures = api_call(league_fixtures)

In [68]:
first_game = league_fixtures['api']['fixtures'][0]

In [69]:
first_game

{'fixture_id': 157015,
 'league_id': 524,
 'event_date': '2019-08-09T19:00:00+00:00',
 'event_timestamp': 1565377200,
 'firstHalfStart': 1565377200,
 'secondHalfStart': 1565380800,
 'round': 'Regular Season - 1',
 'status': 'Match Finished',
 'statusShort': 'FT',
 'elapsed': 90,
 'venue': 'Anfield (Liverpool)',
 'referee': 'Michael Oliver, England',
 'homeTeam': {'team_id': 40,
  'team_name': 'Liverpool',
  'logo': 'https://media.api-football.com/teams/40.png'},
 'awayTeam': {'team_id': 71,
  'team_name': 'Norwich',
  'logo': 'https://media.api-football.com/teams/71.png'},
 'goalsHomeTeam': 4,
 'goalsAwayTeam': 1,
 'score': {'halftime': '4-0',
  'fulltime': '4-1',
  'extratime': None,
  'penalty': None}}

I thought that it would be a good idea to create classes for various objects related to the data we are storing. 

In [105]:
# Define an object orientated approach for a match
class Match():
    def __init__(self, match_json):
        self.match_json = match_json
        self.home_team = match_json['homeTeam']['team_name']
        self.away_team = match_json['awayTeam']['team_name']
        self.fixture_id = match_json['fixture_id']
        
    def fixture(self):
        return(self.home_team + ' vs. ' + self.away_team)
    
    def final_score(self):
        return(self.match_json['score']['fulltime'])

In [106]:
first_game_obj = Match(first_game)

In [107]:
first_game_obj.fixture()

'Liverpool vs. Norwich'

In [108]:
first_game_obj.final_score()

'4-1'

### Events
We need _{fixture_id}_ to grab the events that the API offers for a given match. 

In [103]:
events_url = base_url + 'events/'

In [111]:
first_game_url = events_url + str(first_game_obj.fixture_id);first_game_url

'https://api-football-v1.p.rapidapi.com/v2/events/157015'

In [115]:
events = api_call(first_game_url)['api']

In [117]:
events['events']

[{'elapsed': 7,
  'team_id': 40,
  'teamName': 'Liverpool',
  'player_id': 19066,
  'player': 'Grant Hanley',
  'assist_id': None,
  'assist': None,
  'type': 'Goal',
  'detail': 'Own Goal'},
 {'elapsed': 19,
  'team_id': 40,
  'teamName': 'Liverpool',
  'player_id': 306,
  'player': 'Mohamed Salah',
  'assist_id': 302,
  'assist': 'Roberto Firmino',
  'type': 'Goal',
  'detail': 'Normal Goal'},
 {'elapsed': 28,
  'team_id': 40,
  'teamName': 'Liverpool',
  'player_id': 290,
  'player': 'Virgil van Dijk',
  'assist_id': 306,
  'assist': 'Mohamed Salah',
  'type': 'Goal',
  'detail': 'Normal Goal'},
 {'elapsed': 39,
  'team_id': 40,
  'teamName': 'Liverpool',
  'player_id': 280,
  'player': 'Alisson',
  'assist_id': 18812,
  'assist': 'Adrián',
  'type': 'subst',
  'detail': 'Adrián'},
 {'elapsed': 42,
  'team_id': 40,
  'teamName': 'Liverpool',
  'player_id': 305,
  'player': 'Divock Origi',
  'assist_id': 283,
  'assist': 'Trent Alexander-Arnold',
  'type': 'Goal',
  'detail': 'Normal

Create a class that allows for easily pulling the goals from an _Events_ response. 

In [208]:
import numpy as np

class Events:
    def __init__(self, match_events_json):
        self.n_events = match_events_json['results']
        self.events = match_events_json['events']
        events_df = pd.DataFrame.from_dict(match_events_json['events'])
        self.events_df = events_df
        
    def goals(self):
        goals = self.events_df[self.events_df['type'] == 'Goal'][['type', 'detail', 'player', 'teamName', 'elapsed']].set_index('elapsed')
        goals.index.rename('mins', inplace=True)
        return(goals)

In [211]:
Events(events).goals()

,type,detail,player,teamName
mins,,,,
7,Goal,Own Goal,Grant Hanley,Liverpool
19,Goal,Normal Goal,Mohamed Salah,Liverpool
28,Goal,Normal Goal,Virgil van Dijk,Liverpool
42,Goal,Normal Goal,Divock Origi,Liverpool
64,Goal,Normal Goal,Teemu Pukki,Norwich


In [271]:
stats = api_call(base_url + 'statistics/fixture/{0}'.format(first_game_obj.fixture_id))

In [272]:
stats['api']['statistics']

{'Shots on Goal': {'home': '7', 'away': '5'},
 'Shots off Goal': {'home': '5', 'away': '6'},
 'Total Shots': {'home': '15', 'away': '12'},
 'Blocked Shots': {'home': '3', 'away': '1'},
 'Shots insidebox': {'home': '11', 'away': '8'},
 'Shots outsidebox': {'home': '4', 'away': '4'},
 'Fouls': {'home': '9', 'away': '9'},
 'Corner Kicks': {'home': '11', 'away': '2'},
 'Offsides': {'home': '0', 'away': '5'},
 'Ball Possession': {'home': '58%', 'away': '42%'},
 'Yellow Cards': {'home': '0', 'away': '2'},
 'Red Cards': {'home': None, 'away': None},
 'Goalkeeper Saves': {'home': '4', 'away': '4'},
 'Total passes': {'home': '477', 'away': '361'},
 'Passes accurate': {'home': '387', 'away': '277'},
 'Passes %': {'home': None, 'away': None}}

Start working on the _Stats_ object that produces a `pd.DataFrame` for the stats from a match. 

In [273]:
class Stats:
    def __init__(self, stats_json):
        self.stats_json_clean = str(stats_json).replace("%", "")
        self.stats_json = stats_json
        
    def df(self):
        return(pd.DataFrame.from_dict(eval(self.stats_json_clean)))

In [278]:
Stats(stats['api']['statistics']).df().fillna(value=pd.np.nan)

,Shots on Goal,Shots off Goal,Total Shots,Blocked Shots,Shots insidebox,Shots outsidebox,Fouls,Corner Kicks,Offsides,Ball Possession,Yellow Cards,Red Cards,Goalkeeper Saves,Total passes,Passes accurate,Passes
away,5,6,12,1,8,4,9,2,5,42,2,NaN,4,361,277,NaN
home,7,5,15,3,11,4,9,11,0,58,0,NaN,4,477,387,NaN


In [275]:
Stats(stats['api']['statistics']).stats_json

{'Shots on Goal': {'home': '7', 'away': '5'},
 'Shots off Goal': {'home': '5', 'away': '6'},
 'Total Shots': {'home': '15', 'away': '12'},
 'Blocked Shots': {'home': '3', 'away': '1'},
 'Shots insidebox': {'home': '11', 'away': '8'},
 'Shots outsidebox': {'home': '4', 'away': '4'},
 'Fouls': {'home': '9', 'away': '9'},
 'Corner Kicks': {'home': '11', 'away': '2'},
 'Offsides': {'home': '0', 'away': '5'},
 'Ball Possession': {'home': '58%', 'away': '42%'},
 'Yellow Cards': {'home': '0', 'away': '2'},
 'Red Cards': {'home': None, 'away': None},
 'Goalkeeper Saves': {'home': '4', 'away': '4'},
 'Total passes': {'home': '477', 'away': '361'},
 'Passes accurate': {'home': '387', 'away': '277'},
 'Passes %': {'home': None, 'away': None}}

In [303]:
testing = stats['api']['statistics']

In [304]:
for key, value in testing.items():
    for value_key, value_value in value.items():
        if value_value != None and value_value != np.nan:
            testing[key][value_key] = str(value_value).replace('%', '')
        else:
            testing[key][value_key] = np.NaN

In [305]:
testing

{'Shots on Goal': {'home': '7', 'away': '5'},
 'Shots off Goal': {'home': '5', 'away': '6'},
 'Total Shots': {'home': '15', 'away': '12'},
 'Blocked Shots': {'home': '3', 'away': '1'},
 'Shots insidebox': {'home': '11', 'away': '8'},
 'Shots outsidebox': {'home': '4', 'away': '4'},
 'Fouls': {'home': '9', 'away': '9'},
 'Corner Kicks': {'home': '11', 'away': '2'},
 'Offsides': {'home': '0', 'away': '5'},
 'Ball Possession': {'home': '58', 'away': '42'},
 'Yellow Cards': {'home': '0', 'away': '2'},
 'Red Cards': {'home': 'nan', 'away': 'nan'},
 'Goalkeeper Saves': {'home': '4', 'away': '4'},
 'Total passes': {'home': '477', 'away': '361'},
 'Passes accurate': {'home': '387', 'away': '277'},
 'Passes %': {'home': 'nan', 'away': 'nan'}}